In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

import requests
import json
import geopandas as gpd

# WeGo Public Transit

[WeGo Public Transit](https://www.wegotransit.com/) is a public transit system serving the Greater Nashville
and Davidson County area. WeGo provides local and regional bus routes, the WeGo Star train service connecting
Lebanon to downtown Nashville, along with several other transit services.

In this project, your goal will be to analyze data from WeGo's bus routes and attempt to identify intersections
and other locations which experience frequent delays. This analysis can look by route, by direction of travel,
and by day or day of the week.

You can also consider time of day in your investigation (for example, comparing
the AM and PM rush on Monday through Friday).

You can obtain the data for this project through Swiftly. You will be provided with an API key to  access this data.
The API that you will be using is the [Speed Map API](https://swiftly-inc.stoplight.io/docs/standalone/573bc3fb95867-speed-map),
which provides average speed information for a single route and direction over a given date range.

Note that the API can return either speed data at the stop segment level or return high resolution data,
which provides information for stop segments of approximately 25 meters along with more detailed speed and duration information.

Please take the time to carefully read the API documentation so that you are familiar with the query parameter option
and understand the response data.

You can get more information about these routes, such as route numbers and stop information,
using the [Agency Routes API](https://swiftly-inc.stoplight.io/docs/standalone/5c0bd000bb0f2-agency-routes).
A user-friendly map showing all routes is available at https://www.wegotransit.com/ride/maps-schedules/bus/. 

If time allows, investigate all routes, but start with the high ridership and frequent routes:
* 3: West End
* 7: Hillsboro
* 22: Bordeaux
* 23: Dickerson Pike
* 50: Charlotte Pike
* 52: Nolensville Pike
* 55: Murfreesboro Pike
* 56: Gallatin Pike

Bottlenecks - constant/irregular
(constant)
*ignoring bus stop sections
Finding downtown vs suburbs bottlenecks
1st Quartile of each route for constant
*Delayed data

Focus on Inbound/Outbound
(inbound)

Time categories -
(6am - 9am AM Peak)
9am-230pm Off Peak
230pm-6pm PM Peak
6pm-6am Overnight
Day of Interest
Saturday
Sunday
Holidays*
(M/F)
(Tues-Thurs)
(5/25-7/26)
(1/25-5/24)


# API access
Agency Info: agency_details, routes, schedule

Speed-map: Check all, speed-map



In [2]:
with open('../data/WeGo.json') as fi:
    credentials = json.load(fi)

In [3]:
Authorization = credentials['Authorization']

In [4]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params_geo3 = {
    'direction': '1',
    'startDate': '01-07-2022',
    'endDate': '01-26-2022',
    'daysOfWeek': '1,2,3,4,5',
    'begintime': '06:00',
    'endtime': '09:00',
    #'excludedates': '01-17-2022,02-18-2022,03-14-2022,03-15,2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022',
    'format': 'geojson',
    'resolution': 'hiRes'
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

response_geo3 = requests.get(URL, params = params_geo3, headers = headers)
response_geo3

# dig into json to get list of line-segment dicts
segments3 = response_geo3.json()['features']

# convert to GeoDataFrame the quick way
fullweekgeo = gpd.GeoDataFrame.from_features(segments3)

fullweekgeo

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,numTrips
0,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,2.099551,11.196203,155.210,2.337,1.894,1.758,1.6150,1.503,1.353,0.900,NaN
1,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,3.201034,7.662212,369.046,4.105,3.525,3.009,2.4800,2.099,1.760,0.688,NaN
2,"LINESTRING (-86.79316 36.16427, -86.79291 36.1...",MLK,Martin Luther King School,MCC5_11,Central 5th Ave - Bay 11,24.969,2.665333,9.368060,6.451,6.451,5.218,3.362,2.8375,0.529,0.173,0.173,NaN
3,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.648047,5.256616,62.613,7.831,6.125,4.941,3.6435,3.154,2.521,0.973,NaN
4,"LINESTRING (-86.88358 36.14213, -86.88338 36.1...",ANNCHAWM,Annex Ave & Charlotte Pike Wb,PREWILNN,Premier Dr & Wilclay Dr Nb,24.837,2.842693,8.737138,39.056,3.294,3.026,2.860,2.5880,2.315,2.080,0.919,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,"LINESTRING (-86.82427 36.15280, -86.82401 36.1...",CXISYLHE,37th Ave/Sylvan Heights Station Inbound,CHA33AEN,Charlotte Ave & 33rd Ave N,23.427,1.530515,15.306612,6.690,1.789,1.670,1.595,1.4930,1.387,1.259,0.718,NaN
910,"LINESTRING (-86.78612 36.16793, -86.78590 36.1...",MLK,Martin Luther King School,MCC5_11,Central 5th Ave - Bay 11,24.969,2.605833,9.581965,4.557,4.557,4.325,3.826,2.3895,2.165,0.173,0.173,NaN
911,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.505134,9.844583,14.629,3.590,3.001,2.662,2.2440,2.008,1.747,0.903,NaN
912,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,2.500923,9.660833,60.655,3.312,2.460,2.098,1.8490,1.685,1.514,1.058,NaN


In [5]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params_geo1 = {
    'direction': '1',
    'startDate': '01-07-2022',
    'endDate': '01-26-2022',
    'daysOfWeek': '2,3,4',
    'begintime': '06:00',
    'endtime': '09:00',
    #'excludedates': '01-17-2022,02-18-2022,03-14-2022,03-15,2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022',
    'format': 'geojson',
    'resolution': 'hiRes'
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

response_geo1 = requests.get(URL, params = params_geo1, headers = headers)
response_geo1

# dig into json to get list of line-segment dicts
segments1 = response_geo1.json()['features']

# convert to GeoDataFrame the quick way
twtgeo = gpd.GeoDataFrame.from_features(segments1)

In [6]:
twtgeo.shape

(914, 17)

In [7]:
twtgeo

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,numTrips
0,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.889535,12.440627,47.753,2.193,1.848,1.735,1.6095,1.497,1.349,0.900,NaN
1,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.735523,8.966110,8.071,4.049,3.467,2.989,2.4930,2.098,1.761,0.688,NaN
2,"LINESTRING (-86.79316 36.16427, -86.79291 36.1...",MLK,Martin Luther King School,MCC5_11,Central 5th Ave - Bay 11,24.969,3.127000,7.984970,6.451,6.451,6.451,4.910,2.8660,1.345,0.325,0.325,NaN
3,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.667920,5.234237,62.613,7.772,6.209,4.964,3.6620,3.172,2.578,0.973,NaN
4,"LINESTRING (-86.88358 36.14213, -86.88338 36.1...",ANNCHAWM,Annex Ave & Charlotte Pike Wb,PREWILNN,Premier Dr & Wilclay Dr Nb,24.837,2.846487,8.725492,39.056,3.293,3.014,2.843,2.5860,2.305,2.061,0.919,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,"LINESTRING (-86.85218 36.15152, -86.85192 36.1...",CXIWTBRG,White Bridge Station Inbound,CXI53AVE,53rd Ave Station Inbound,24.864,2.502626,9.935164,16.677,3.707,2.584,2.327,2.0195,1.811,1.570,1.229,NaN
910,"LINESTRING (-86.78612 36.16793, -86.78590 36.1...",MLK,Martin Luther King School,MCC5_11,Central 5th Ave - Bay 11,24.969,3.328500,7.501577,4.557,4.557,4.557,4.268,3.2180,2.390,2.321,2.321,NaN
911,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.464563,10.006642,10.836,3.526,2.915,2.601,2.2390,2.000,1.746,0.903,NaN
912,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,2.677554,9.023534,60.655,3.411,2.534,2.126,1.8620,1.685,1.532,1.058,NaN


In [8]:
#rt50geo['data']['segments'][3:4]

In [9]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params_geo2 = {
    'direction': '1',
    'startDate': '01-07-2022',
    'endDate': '01-26-2022',
    'daysOfWeek': '1,5',
    'begintime': '06:00',
    'endtime': '09:00',
    #'excludedates': '01-17-2022,02-18-2022,03-14-2022,03-15,2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022',
    'format': 'geojson',
    'resolution': 'hiRes'
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

response_geo2 = requests.get(URL, params = params_geo2, headers = headers)
response_geo2

# dig into json to get list of line-segment dicts
segments2 = response_geo2.json()['features']

# convert to GeoDataFrame the quick way
mfgeo = gpd.GeoDataFrame.from_features(segments2)

In [10]:
mfgeo.shape

(909, 17)

In [11]:
mfgeo

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,numTrips
0,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,2.455633,9.572685,155.210,2.570,1.963,1.806,1.6300,1.515,1.365,1.095,NaN
1,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,3.946754,6.214474,369.046,4.175,3.589,3.075,2.4800,2.110,1.757,1.127,NaN
2,"LINESTRING (-86.79316 36.16427, -86.79291 36.1...",MLK,Martin Luther King School,MCC5_11,Central 5th Ave - Bay 11,24.969,1.742000,14.333525,3.311,3.311,3.311,3.311,1.7420,0.173,0.173,0.173,NaN
3,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.610970,5.298885,43.107,8.007,5.881,4.924,3.5810,3.090,2.452,1.810,NaN
4,"LINESTRING (-86.88358 36.14213, -86.88338 36.1...",ANNCHAWM,Annex Ave & Charlotte Pike Wb,PREWILNN,Premier Dr & Wilclay Dr Nb,24.837,2.835603,8.758984,22.437,3.319,3.097,2.903,2.5930,2.336,2.102,1.040,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,"LINESTRING (-86.82427 36.15280, -86.82401 36.1...",CXISYLHE,37th Ave/Sylvan Heights Station Inbound,CHA33AEN,Charlotte Ave & 33rd Ave N,23.427,1.537447,15.237598,6.690,1.809,1.666,1.599,1.4870,1.380,1.262,0.792,NaN
905,"LINESTRING (-86.78612 36.16793, -86.78590 36.1...",MLK,Martin Luther King School,MCC5_11,Central 5th Ave - Bay 11,24.969,1.160500,21.515726,2.148,2.148,2.148,2.148,1.1605,0.173,0.173,0.173,NaN
906,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.573952,9.581375,14.629,3.632,3.165,2.728,2.2550,2.036,1.749,1.432,NaN
907,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,2.217398,10.896104,12.396,3.215,2.371,2.079,1.8380,1.685,1.503,1.253,NaN


In [12]:
mtwtf = pd.concat([mfgeo, twtgeo])

In [13]:
mtwtf

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,numTrips
0,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,2.455633,9.572685,155.210,2.570,1.963,1.806,1.6300,1.515,1.365,1.095,NaN
1,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,3.946754,6.214474,369.046,4.175,3.589,3.075,2.4800,2.110,1.757,1.127,NaN
2,"LINESTRING (-86.79316 36.16427, -86.79291 36.1...",MLK,Martin Luther King School,MCC5_11,Central 5th Ave - Bay 11,24.969,1.742000,14.333525,3.311,3.311,3.311,3.311,1.7420,0.173,0.173,0.173,NaN
3,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.610970,5.298885,43.107,8.007,5.881,4.924,3.5810,3.090,2.452,1.810,NaN
4,"LINESTRING (-86.88358 36.14213, -86.88338 36.1...",ANNCHAWM,Annex Ave & Charlotte Pike Wb,PREWILNN,Premier Dr & Wilclay Dr Nb,24.837,2.835603,8.758984,22.437,3.319,3.097,2.903,2.5930,2.336,2.102,1.040,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,"LINESTRING (-86.85218 36.15152, -86.85192 36.1...",CXIWTBRG,White Bridge Station Inbound,CXI53AVE,53rd Ave Station Inbound,24.864,2.502626,9.935164,16.677,3.707,2.584,2.327,2.0195,1.811,1.570,1.229,NaN
910,"LINESTRING (-86.78612 36.16793, -86.78590 36.1...",MLK,Martin Luther King School,MCC5_11,Central 5th Ave - Bay 11,24.969,3.328500,7.501577,4.557,4.557,4.557,4.268,3.2180,2.390,2.321,2.321,NaN
911,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.464563,10.006642,10.836,3.526,2.915,2.601,2.2390,2.000,1.746,0.903,NaN
912,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,2.677554,9.023534,60.655,3.411,2.534,2.126,1.8620,1.685,1.532,1.058,NaN


In [14]:
type(mtwtf)

geopandas.geodataframe.GeoDataFrame

In [15]:
def concat_frames(geo_response, response):
    """This function pulls in two API requests, puts one in a geodataframe the other in a dataframe and concats desired columns"""
    geo_df = gpd.GeoDataFrame.from_features(geo_response.json()['features'])
    res_df = pd.DataFrame(response.json()['data']['segments'])
    full_df = pd.concat([geo_df, res_df[['countForSegment', 'percent20Secs']]], axis = 1)
    return full_df

In [16]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '01-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,5',
    'endDate' : '02-24-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '01-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,5',
    'endDate' : '02-24-2022',
    'endTime' : '09:00',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [17]:
janfebm = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

In [18]:
janfebm

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.187511,11.142801,9.778,3.249,2.160,1.962,1.6895,1.542,1.410,1.285,94,1.473
1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.506204,15.606784,2.101,1.784,1.666,1.601,1.4840,1.370,1.260,1.154,93,1.332
2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.286516,10.798525,5.661,2.907,2.636,2.314,2.1260,1.992,1.803,1.280,93,1.901
3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.827600,8.674141,15.590,4.361,3.559,2.811,2.3130,1.978,1.705,0.969,95,1.844
4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.251321,5.747155,27.961,6.624,5.361,4.611,3.4230,2.916,2.366,1.326,81,2.647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.590707,9.519409,6.651,3.637,3.116,2.878,2.3515,2.075,1.750,1.368,92,1.975
562,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.829427,13.206868,3.529,2.389,2.085,1.906,1.6920,1.580,1.407,1.248,96,1.491
563,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,7.637885,3.177581,48.793,17.724,15.650,8.757,4.6025,1.976,1.599,1.249,96,1.780
564,"LINESTRING (-86.83597 36.15238, -86.83570 36.1...",CXISYLPA,42nd Ave/Sylvan Park Station Inbound,CHA39AEF,Charlotte Ave & 39th Ave N Eb,23.766,3.541189,6.711305,7.141,5.666,4.612,4.046,3.2780,2.658,1.933,0.990,95,2.345


In [19]:
type(janfebm)

geopandas.geodataframe.GeoDataFrame

In [20]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '02-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,5',
    'endDate' : '03-24-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '02-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,5',
    'endDate' : '03-24-2022',
    'endTime' : '09:00',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [21]:
febmar = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

In [22]:
febmar

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.260596,10.782555,6.486,4.875,2.417,2.004,1.7160,1.570,1.460,1.273,94,1.526
1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.544543,15.219389,7.156,1.726,1.673,1.594,1.4740,1.357,1.228,1.006,92,1.312
2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.257000,10.939743,3.425,2.976,2.623,2.422,2.1920,1.957,1.743,1.446,91,1.854
3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.688221,9.123878,8.830,4.692,3.398,2.861,2.2000,1.950,1.712,1.350,95,1.818
4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.064935,6.010674,9.621,6.448,5.212,4.009,3.4930,3.154,2.678,1.577,93,3.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.818699,8.749427,9.921,3.775,3.436,3.057,2.7010,2.268,1.749,0.921,93,2.102
564,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.977261,12.219429,4.558,2.747,2.388,2.025,1.8250,1.613,1.462,1.244,92,1.557
565,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,6.080189,3.991652,50.733,21.648,6.578,3.182,2.1050,1.814,1.587,1.227,95,1.696
566,"LINESTRING (-86.83597 36.15238, -86.83570 36.1...",CXISYLPA,42nd Ave/Sylvan Park Station Inbound,CHA39AEF,Charlotte Ave & 39th Ave N Eb,23.766,3.796832,6.259429,10.443,6.140,5.170,4.274,3.3610,2.676,2.179,1.819,95,2.456


In [23]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '03-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,5',
    'endDate' : '04-24-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '03-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,5',
    'endDate' : '04-24-2022',
    'endTime' : '09:00',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [24]:
marapr = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

In [25]:
marapr

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.716505,8.972927,8.334,5.873,3.988,2.874,1.7830,1.550,1.414,0.879,105,1.467
1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.481139,15.870894,2.306,1.757,1.666,1.583,1.4810,1.365,1.230,0.910,101,1.290
2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.335961,10.569954,3.802,3.125,2.732,2.490,2.2670,2.036,1.760,1.463,102,1.937
3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.505733,9.788353,5.935,3.943,3.004,2.681,2.1770,1.937,1.674,1.390,105,1.839
4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.995311,4.891187,84.397,6.435,5.756,4.437,3.4925,3.176,2.452,2.066,106,2.868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.641162,9.337557,6.021,3.718,3.263,2.893,2.4180,2.140,1.822,1.282,105,2.016
562,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,2.012720,12.004154,5.049,3.086,2.165,1.967,1.8030,1.665,1.482,1.221,107,1.591
563,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,6.805404,3.566284,41.366,23.447,10.668,5.375,2.0650,1.788,1.638,1.151,104,1.721
564,"LINESTRING (-86.83597 36.15238, -86.83570 36.1...",CXISYLPA,42nd Ave/Sylvan Park Station Inbound,CHA39AEF,Charlotte Ave & 39th Ave N Eb,23.766,4.021457,5.909798,11.045,6.298,5.274,4.616,3.6050,2.893,2.183,1.812,105,2.416


In [26]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '04-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,5',
    'endDate' : '05-24-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '04-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,5',
    'endDate' : '05-24-2022',
    'endTime' : '09:00',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [27]:
aprmay = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

In [28]:
aprmay

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.641241,9.228616,7.777,5.218,3.840,2.922,1.7605,1.572,1.492,1.365,108,1.516
1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.468255,16.010162,2.623,1.679,1.576,1.533,1.4695,1.372,1.265,0.969,102,1.322
2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.362366,10.451810,5.321,3.055,2.732,2.424,2.2410,2.025,1.806,1.348,101,1.921
3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.673546,9.173958,6.087,4.512,3.514,2.883,2.2910,2.020,1.774,1.429,108,1.883
4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.337798,5.632581,10.482,6.697,5.950,4.810,3.5990,3.253,2.539,1.779,104,3.077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.487058,9.916134,4.241,3.768,3.060,2.688,2.1995,2.040,1.797,1.636,104,1.946
562,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,1.916565,12.606408,5.756,2.554,1.985,1.895,1.7340,1.627,1.445,1.149,108,1.567
563,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,6.320278,3.840021,84.840,17.758,8.453,2.633,2.1155,1.912,1.602,1.184,108,1.706
564,"LINESTRING (-86.83597 36.15238, -86.83570 36.1...",CXISYLPA,42nd Ave/Sylvan Park Station Inbound,CHA39AEF,Charlotte Ave & 39th Ave N Eb,23.766,3.777898,6.290800,9.466,5.860,4.897,4.470,3.4115,2.752,2.200,1.625,108,2.417


In [39]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '01-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '4',
    'endDate' : '02-24-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '01-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '4',
    'endDate' : '02-24-2022',
    'endTime' : '09:00',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [40]:
janfebt = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

In [41]:
janfebt

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
0,"LINESTRING (-86.83200 36.15252, -86.83173 36.1...",CHA39AEF,Charlotte Ave & 39th Ave N Eb,CXISYLHE,37th Ave/Sylvan Heights Station Inbound,24.375,2.398322,10.163356,7.472,4.901,2.403,2.021,1.7420,1.660,1.468,1.291,59,1.543
1,"LINESTRING (-86.86416 36.14888, -86.86390 36.1...",CHABALEF,Charlotte Ave & Balmy Ave Eb,CHALELEM,Charlotte Pike & Lellyett St Eb,23.507,1.544579,15.219034,1.955,1.874,1.751,1.696,1.5410,1.413,1.245,1.157,57,1.312
2,"LINESTRING (-86.79194 36.16148, -86.79169 36.1...",CXI17AVE,17th Ave Station Inbound,CXINGULC,11th Ave / N Gulch Station Inbound,24.691,2.372831,10.405714,4.005,3.111,2.726,2.495,2.2550,2.042,1.919,1.662,59,1.986
3,"LINESTRING (-86.84193 36.15216, -86.84166 36.1...",CXIRICHL,46th Ave/Richland Park Station Inbound,CXISYLPA,42nd Ave/Sylvan Park Station Inbound,24.527,2.753678,8.906996,5.589,4.122,3.327,2.833,2.5790,2.154,1.938,1.565,59,2.110
4,"LINESTRING (-86.87277 36.14445, -86.87258 36.1...",PREWILNN,Premier Dr & Wilclay Dr Nb,CXIAMERI,American Station Inbound,24.433,4.266071,5.727284,17.742,7.267,5.610,4.095,3.4425,3.125,2.415,2.201,56,2.851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,"LINESTRING (-86.87143 36.14546, -86.87124 36.1...",CXIAMERI,American Station Inbound,CHABALEF,Charlotte Ave & Balmy Ave Eb,24.662,2.705661,9.114963,6.096,3.938,3.338,2.987,2.4630,2.102,1.746,1.564,56,1.909
563,"LINESTRING (-86.80505 36.15676, -86.80479 36.1...",CXICENTE,23rd Ave / Centennial Station Inbound,CHA20AEF,Charlotte Ave & 20th Ave Eb,24.161,2.063217,11.710353,6.088,3.282,2.389,2.008,1.7620,1.632,1.483,1.195,60,1.570
564,"LINESTRING (-86.85058 36.15184, -86.85031 36.1...",CXI53AVE,53rd Ave Station Inbound,CXIRICHL,46th Ave/Richland Park Station Inbound,24.270,5.675283,4.276439,49.676,14.555,5.939,3.249,2.2190,1.922,1.600,1.330,60,1.740
565,"LINESTRING (-86.88379 36.14136, -86.88404 36.1...",ANNCHAWM,Annex Ave & Charlotte Pike Wb,PREWILNN,Premier Dr & Wilclay Dr Nb,24.837,3.672724,6.762556,5.517,5.050,4.602,4.206,3.6350,3.164,2.505,0.641,58,2.850


In [42]:
janfebm.describe()

,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
count,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000
mean,24.453348,3.734353,10.655819,20.032348,7.623193,5.569251,4.070813,2.601087,2.002016,1.641385,1.255412,93.293286,1.806776
std,0.374216,13.390637,3.962542,63.225275,48.274292,35.886858,19.730647,5.229004,1.781598,0.574417,0.506620,5.415042,0.749665
min,23.427000,1.474842,0.080867,1.926000,1.735000,1.579000,1.507000,1.400500,1.310000,1.174000,0.779000,7.000000,1.229000
25%,24.270000,1.741446,7.838033,3.909500,2.078500,1.865750,1.765250,1.634625,1.524500,1.354000,1.092000,93.000000,1.446500
50%,24.486000,2.239484,10.992863,6.727500,2.918500,2.449000,2.185000,1.938500,1.725500,1.510500,1.221000,95.000000,1.625000
75%,24.691000,3.129291,14.053757,14.887000,4.624750,3.623250,3.086500,2.441375,2.097750,1.757000,1.323000,96.000000,1.952000
max,24.929000,307.419000,16.488145,1126.582000,1126.582000,838.827000,455.697000,107.538000,39.379000,11.292000,11.292000,97.000000,13.724000


In [43]:
janfebt.describe()

,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs
count,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000
mean,24.410220,3.751136,9.809653,16.517644,6.972721,5.025822,3.929286,2.736904,2.204392,1.872108,1.507884,58.640212,2.015737
std,1.092888,7.896097,3.437733,27.684023,18.009743,16.178879,11.911102,6.168996,4.176646,3.780708,3.790980,3.601777,3.782803
min,0.000000,1.522220,0.000000,1.889000,1.785000,1.643000,1.563000,1.459500,1.357000,1.222000,0.641000,1.000000,1.297000
25%,24.270000,1.964452,7.636102,4.061000,2.425500,1.989000,1.854500,1.722000,1.574000,1.411500,1.186000,58.000000,1.504000
50%,24.486000,2.379862,10.297613,7.152000,3.316000,2.576000,2.316000,2.026000,1.795000,1.576000,1.320000,59.000000,1.704000
75%,24.691000,3.229458,12.436107,15.903500,5.019500,3.682500,3.157500,2.497500,2.143500,1.832500,1.488000,60.000000,2.022000
max,24.929000,144.449500,15.406849,331.781000,331.781000,331.781000,242.887000,114.886500,90.003000,90.003000,90.003000,60.000000,90.003000
